In [ ]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import multiprocessing as mp

### Instance routing
This notebook takes the depot and all parcels for one instance and performs a parallelized N:N routing to generate a distance and travel time matrix.

In [ ]:
# Manage inputs and outputs
graph_path = "../../results/network/without_zfe.graphml"
depots_path = "../../results/depots/individual/laposte/96.gpkg"
parcels_path = "../../results/parcels/per_depot/baseline_2022/laposte/96.gpkg"
output_path = "/home/shoerl/temp/test.nc"
processes = 3

if "snakemake" in locals():
    graph_path = snakemake.input["graph"]
    depots_path = snakemake.input["depots"]
    parcels_path = snakemake.input["parcels"]
    output_path = snakemake.output[0]
    # processes = snakemake.threads

In [ ]:
# Load network graph
graph = ox.load_graphml(graph_path)

In [ ]:
# Load parcels
df_parcels = gpd.read_file(parcels_path)

assert len(df_parcels["depot_id"].unique()) == 1
depot_id = df_parcels["depot_id"].unique()[0]

In [ ]:
# Load depots and select the relevant one
df_depots = gpd.read_file(depots_path)
df_depots = df_depots[df_depots["depot_id"] == depot_id]
assert len(df_depots) == 1

In [ ]:
# Find all relevant nodes
nodes = set(df_parcels["node"].unique()) | set(df_depots["node"].unique())
nodes = [node for node in nodes if node in graph.nodes]

In [ ]:
# Parallel routing of all distances requires in the instance
def initializer(nodes, graph):
    globals()["nodes"] = nodes
    globals()["graph"] = graph 
    globals()["edge_lengths"] = nx.get_edge_attributes(graph, "length")
    globals()["edge_travel_times"] = nx.get_edge_attributes(graph, "travel_time")

def process(node_index):
    source_node = nodes[node_index]
    routes = nx.single_source_dijkstra_path(graph, source_node, weight = "travel_time")

    row = xr.DataArray(
        dims = ("origin", "destination", "attribute"),
        coords = { 
            "origin": [source_node], "destination": nodes, 
            "attribute": ["travel_time", "distance"] }
    )

    for target_node in nodes:
        route = routes[target_node]
        row.loc[source_node, target_node, "travel_time"] = nx.path_weight(graph, route, "travel_time")
        row.loc[source_node, target_node, "distance"] = nx.path_weight(graph, route, "length")
    
    return row

if processes == 1:
    initializer(nodes, graph)
    matrix = [process(index) for index in tqdm(range(len(nodes)))]

else:
    with mp.Pool(processes, initializer, (nodes, graph)) as pool:
        matrix = [row for row in 
            tqdm(pool.imap_unordered(process, range(len(nodes))), total = len(nodes))
        ]

if len(nodes) > 0:
    matrix = xr.concat(matrix, "origin").sortby(["origin", "destination", "attribute"])
else:
    matrix = xr.DataArray(np.zeros((0,0,2)), dims = ("origin", "destination", "attribute"), coords = {
        "origin": np.zeros((0,), dtype=np.int64), "destination": np.zeros((0,), dtype=np.int64), "attribute": ["travel_time", "distance"]
    })

In [ ]:
assert set(matrix.coords["origin"].values) == set(nodes)
assert set(matrix.coords["destination"].values) == set(nodes)

In [ ]:
# Output
matrix.to_netcdf(output_path)